## RDD

#### Transformations:

https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations

#### Actions

https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions

In [ ]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().appName("Sesion_3").master("local[*]").getOrCreate()
val sc = spark.sparkContext

In [ ]:
sc.uiWebUrl

In [ ]:
import org.apache.spark.rdd.RDD
val pokemon_rdd: RDD[String] = sc.textFile("../../resources/data/csv/pokemon.csv")
    //.flatMap(line => (1 to 10).map(item => line + "," + item.toString))
pokemon_rdd.collect().foreach(item => println(item))

In [ ]:
val clean_pokemon_rdd: RDD[Seq[Any]] = pokemon_rdd
    .map(line => line.split(","))
    .filter(array_str => array_str(0).matches("^[0-9]+$"))
    .map(array_str => Seq(array_str(0).toInt, array_str(1), array_str(2), array_str(3),
                          array_str(4).toInt, array_str(5).toInt, array_str(6).toInt, array_str(7).toInt,
                          array_str(8).toInt, array_str(9).toInt, array_str(10).toInt, array_str(11).toInt,
                          array_str(12).matches("True"))).persist()

clean_pokemon_rdd.collect().foreach(item => println(item))

In [ ]:
// Cuantos pokemon legendarios existen?
val count_legendary: Int = clean_pokemon_rdd
    .map(array => (array(12), 1))
    .filter(item => item._1 == true)
    .map(item => item._2)
    .reduce((item_1, item_2) => item_1 + item_2)

//println("count_legendary", count_legendary)

val count_legendary_2: Int = clean_pokemon_rdd
    .map(array => (array(12), 1))
    .reduceByKey((item_1, item_2) => item_1 + item_2)
    .filter(item => item._1 == true)
    .collect()(0)._2

//print("count_legendary_2", count_legendary_2)

In [ ]:
// Valor promedio de ataque por cada type_1
clean_pokemon_rdd
    .map(array => (array(2), array(6).toString.toFloat))
    .groupByKey()
    .mapValues(values => (values.sum, values.size))
    .mapValues(items => items._1 / items._2)
    .collect()
    .foreach(item => println(item))

println("+-+-+")

clean_pokemon_rdd
    .map(array => (array(2), array(6).toString.toFloat))
    .groupByKey()
    .map(values => (values._1, (values._2.sum / values._2.size)))
    .collect()
    .foreach(item => println(item))

println("+-+-+")

clean_pokemon_rdd
    .map(array => (array(2), (array(6).toString.toFloat, 1)))
    .reduceByKey((tuple_1, tuple_2) => (tuple_1._1 + tuple_2._1, tuple_1._2 + tuple_2._2))
    .map(results => (results._1, results._2._1 / results._2._2))
    .collect()
    .foreach(item => println(item))

In [ ]:
// Pokemon con mejor score en Total POR cada TYPE 1 Y POR GENERACIÓN OMITIENDO LOS LEGENDARIOS
clean_pokemon_rdd
    .filter(array => array(12) == false)
    .map(array => ((array(2), array(11)), (array(1), array(4).toString.toInt)))
    .reduceByKey((item_1, item_2) => if (item_1._2 > item_2._2) item_1 else item_2)
    .collect()
    .foreach(item => println(item))

In [ ]:
// Top 3 Pokemon con mejor score en Total POR cada TYPE 1 Y POR GENERACIÓN OMITIENDO LOS LEGENDARIOS
clean_pokemon_rdd
    .filter(array => array(12) == false)
    .map(array => ((array(2), array(11)), (array(1), array(4).toString.toInt)))
    .groupByKey()
    .mapValues(values => values.toSeq.sortBy(item => item._2)(Ordering[Int].reverse).take(3))
    .flatMapValues(results => results)
    .collect()
    .foreach(item => println(item))